# Geomechanical Injection Scenario Toolkit (GIST)

#Disclaimer
GIST aims to give the _gist_ of a wide range of potential scenarios and aid collective decision making when responding to seismicity.

The results of GIST are entirely dependent upon the inputs provided, which may be incomplete or inaccurate.

There are other potentially plausible inducement scenarios that are not considered, including fluid migration into the basement, 
out-of-zone poroelastic stressing, or hydraulic fracturing.

None of the individual models produced by GIST accurately represent what happens in the subsurface and cannot be credibly used 
to accurately assign liability or responsibility for seismicity.

"All models are wrong, but some are useful" - George Box, 1976

## Prerequisites

Assumes InjectionSQLScheduled completed successfully and injection data are sampled uniformly in time

##Install Dependencies
- geopandas
- gistMC.py
- eqSQL.py
- gistPlots.py
- numpy
- scipy
- pandas


In [0]:
%restart_python

In [0]:
%run "/Workspace/_utils/Utility_Functions"

In [0]:
!pip install geopandas
!pip install geodatasets
!pip install contextily
#! pip install folium matplotlib mapclassify contextily

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.9/23.9 MB 101.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 155.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 112.3 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 3.7 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 71.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

##Paths

In [0]:
# Paths
homePath='/Workspace/Users/bill.curry@exxonmobil.com/'
# Injection data path 
injPath=homePath+'injection/WeeklyRun/ScheduledOutput/'
# GIST library path
gistPath=homePath+'GIST/'

##Libraries

- numpy
- scipy
- pandas
- matplotlib
- geopandas
- pyspark


In [0]:
import sys
sys.path.append(gistPath+'lib')

In [0]:
#Databricks-specific
#import dataBricksConfig as db
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("eqSQL").getOrCreate()

In [0]:
import numpy as np
import pandas as pd
import os
import gistMC as gi
import eqSQL as es
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas
#import contextily as cx

In [0]:
import gistPlots as gp

#1. Select Event

In [0]:
eventID='texnet2024oqfb'
forecastYears=5.

In [0]:
runPath=gistPath+'/runs/'+eventID+'/'
os.makedirs(runPath, exist_ok=True)

In [0]:
eqs=es.eqSQL()
EQDF=eqs.getEarthquake(eventID)
EQDF=EQDF.rename(columns={'LatitudeErrorKm':'LatitudeError','LongitudeErrorKm':'LongitudeError','EventId':'EventID'})
# What about the other fault plane solution? - 251 / 36 / -97
EQDF['Strike']=80.
EQDF['Dip']=55.
EQDF['Rake']=-85.
# We could make use of the Rake
EQDF.info()
eq=EQDF.loc[0]

getEarthquake:     SeismicEventId  ... B3RecordDeletedUTCDateTime
0         2481836  ...                        NaT

[1 rows x 32 columns] 1
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 38 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   SeismicEventId              1 non-null      int64         
 1   DataSource                  1 non-null      object        
 2   DataSourceUrl               0 non-null      object        
 3   EventID                     1 non-null      object        
 4   EventTimeUtc                1 non-null      datetime64[ns]
 5   EventTimeInLocalTimeZone    1 non-null      object        
 6   EventTimeZone               1 non-null      object        
 7   EventType                   1 non-null      object        
 8   DepthKm                     1 non-null      float64       
 9   DepthErrorKm                1 non-null   

In [0]:
EQDF.to_csv(runPath+'EQ.csv')

#2. Initial Run

##2.1 Subsurface Model Parameters

###Inputs:

In [0]:
# Binary deep/shallow parameter
deepOrShallow='Deep'
# Depth from surface (ft)
# Distinguishes deep/shallow where we don't have a horizon 
depthCutoff=8000.

In [0]:
nRealizations=200

In [0]:
# Water density minimum/maximum (kg/m3)
WaterDensityMin=1015.
WaterDensityMax=1025.
# Water viscosity minimum/maximum (Pa.s)
WaterViscosityMin=0.000799
WaterViscosityMax=0.000801
# Fluid Compressibility minimum/maximum (1/Pa)
FluidCompressibilityMin=0.000000000359
FluidCompressibilityMax=0.000000000361

In [0]:
# Porosity in percent
PorosityPercentMin=3.
PorosityPercentMax=15.

# Permeability in millidarcies
PermMDMin=5.
PermMDMax=500.

# Thickness in feet
ThicknessFTMin=200.
ThicknessFTMax=2000.

# Vertical compressibility minimum/maximum (1/Pa)
VerticalCompressibilityMin=0.00000000107
VerticalCompressibilityMax=0.00000000109

In [0]:
# Verbosity - 0=silent, 1=some, 2=lots
verb=2
# Minimum Pressure change to care about in PSI
dPCutoff=0.5
# Maximum Number of wells to plot
nWells=50
# What is the first year we plot?
minYear=1980

### Set Interval-Specific Paths...


In [0]:
# Set an output directory for this earthquake and this interval
runIntervalPath=runPath+deepOrShallow+'/'
initialRunIntervalPath=runIntervalPath+'initialRun/'
updatedRunIntervalPath=runIntervalPath+'udpatedRun/'
forecastRunIntervalPath=runIntervalPath+'forecastRun/'
disposalPath=runIntervalPath+'updatedDisposal/'
# Make directory if it doesn't exist
os.makedirs(runIntervalPath, exist_ok=True)
os.makedirs(initialRunIntervalPath, exist_ok=True)
os.makedirs(updatedRunIntervalPath, exist_ok=True)
os.makedirs(forecastRunIntervalPath, exist_ok=True)
os.makedirs(disposalPath, exist_ok=True)
os.makedirs(initialRunIntervalPath+'perWell', exist_ok=True)
os.makedirs(updatedRunIntervalPath+'perWell', exist_ok=True)
os.makedirs(forecastRunIntervalPath+'perWell', exist_ok=True)
################################################
# Output prefix for realizations of parameters #
################################################
RealizationPrefix=runIntervalPath+'MC'

In [0]:
# Point to appropriate well and injection files from initial database
if deepOrShallow=='Deep':
  WellFile=injPath+'/deep.csv'
  InjFile=injPath+'/deepReg.csv'
elif deepOrShallow=='Shallow':
  WellFile=injPath+'/shallow.csv'
  InjFile=injPath+'/shallowReg.csv'

In [0]:
# Oversampling of time axis (poroelastic-only)
nTimeBins=21
# Poroelastic parameters - for in-zone poroelasticity in v2
ShearModulusMin=4e9
ShearModulusMax=6e9
PoissonsRatioDrainedMin=0.295
PoissonsRatioDrainedMax=0.305
PoissonsRatioUndrainedMin=0.305
PoissonsRatioUndrainedMax=0.315
BiotsCoefficientMin=0.26
BiotsCoefficientMax=0.36
# Fault parameters (poroelastic-only)
FaultFrictionCoeffMin=0.55
FaultFrictionCoeffMax=0.65
RockFrictionCoeffMin=0.55
RockFrictionCoeffMax=0.65

##2.2 Results

###Compute

In [0]:
gist=gi.gistMC(nReal=nRealizations,
                ntBin=nTimeBins)
gist.initPP(rho0_min=WaterDensityMin,
             rho0_max=WaterDensityMax,
             phi_min=PorosityPercentMin,
             phi_max=PorosityPercentMax,
             kMD_min=PermMDMin,
             kMD_max=PermMDMax,
             h_min=ThicknessFTMin,
             h_max=ThicknessFTMax,
             alphav_min=VerticalCompressibilityMin,
             alphav_max=VerticalCompressibilityMax,
             beta_min=FluidCompressibilityMin,
             beta_max=FluidCompressibilityMax)

In [0]:
gist.addWells(wellFile=WellFile,injFile=InjFile,userWellFile=None,userInjFile=None,verbose=verb)
if verb>0:
  gist.wellDF.info()
  EQDF.info()

 gistMC.addWells: no user wells and injection provided, only default wells/injection
 gistMC.addWells: well file added with  2863  wells
 gistMC.addWells: well columns: Index(['ID', 'InjectionWellId', 'UniqueWellIdentifier', 'UICNumber',
       'APINumber', 'LeaseName', 'Operator', 'OperatorType',
       'OperatorPrincipalCompany', 'OperatorPrincipalCompanyType', 'WellName',
       'WellNumber', 'State', 'Basin', 'County', 'District', 'SRAOrSIR',
       'B3InjectionType', 'B3InjectionStatus', 'RegulatoryInjectionType',
       'PermittedMaxLiquidBPD', 'PermittedMaxLiquidPSIG',
       'PermittedMaxGasMCFPerDay', 'PermittedMaxGasPSIG',
       'PermittedCommercialStatus', 'PermittedIntervalTopFt',
       'PermittedIntervalBottomFt', 'InjectionClass', 'PermitStage',
       'PermittedWellDepthClassification', 'DaysApplicationHasBeenInReview',
       'DaysToPermitApproval', 'PermitIsAmendment',
       'PendingApplicationIsAmendment',
       'PendingApplicationRequestedCommercialStatus',
     

In [0]:
selectedWellsDF,ignoredWellsDF,injDF=gist.findWells(eq,PE=False,responseYears=forecastYears,verbose=verb)
# Isolate wells included only for forecast
currentWellsDF=selectedWellsDF[selectedWellsDF['EncompassingDay']<0.].reset_index(drop=True)
forecastWellsDF=selectedWellsDF[selectedWellsDF['EncompassingDay']>=0.].reset_index(drop=True)
print(selectedWellsDF.shape,currentWellsDF.shape,forecastWellsDF.shape)

 gistMC.findWells: iw= 0  of  2863  start date is  1997-07-08
 gistMC.findWells injectionDays 9881
 gistMC.findWells: iw= 1  of  2863  start date is  nan
 gistMC.findWells injectionDays nan
 gistMC.findWells: iw= 2  of  2863  start date is  1991-01-27
 gistMC.findWells injectionDays 12235
 gistMC.findWells: iw= 3  of  2863  start date is  1952-07-22
 gistMC.findWells injectionDays 26303
 gistMC.findWells: iw= 4  of  2863  start date is  2001-04-16
 gistMC.findWells injectionDays 8503
 gistMC.findWells: iw= 5  of  2863  start date is  1976-01-25
 gistMC.findWells injectionDays 17716
 gistMC.findWells: iw= 6  of  2863  start date is  nan
 gistMC.findWells injectionDays nan
 gistMC.findWells: iw= 7  of  2863  start date is  1977-11-06
 gistMC.findWells injectionDays 17065
 gistMC.findWells: iw= 8  of  2863  start date is  1974-02-01
 gistMC.findWells injectionDays 18439
 gistMC.findWells: iw= 9  of  2863  start date is  1980-06-27
 gistMC.findWells injectionDays 16101
 gistMC.findWells: i

In [0]:
scenarioDF=gist.runPressureScenarios(eq,currentWellsDF,injDF,verbose=1)

 gistMC.runPressureScenarios: Number of wells considered:  277
 gistMC.pressureScenario: well  0  of  277 :  1007324
 gistMC.pressureScenario: well  10  of  277 :  2081279
 gistMC.pressureScenario: well  20  of  277 :  2086897
 gistMC.pressureScenario: well  30  of  277 :  2096093
 gistMC.pressureScenario: well  40  of  277 :  2112042
 gistMC.pressureScenario: well  50  of  277 :  2111988
 gistMC.pressureScenario: well  60  of  277 :  2117885
 gistMC.pressureScenario: well  70  of  277 :  2121094
 gistMC.pressureScenario: well  80  of  277 :  2126182
 gistMC.pressureScenario: well  90  of  277 :  2051992
 gistMC.pressureScenario: well  100  of  277 :  2070385
 gistMC.pressureScenario: well  110  of  277 :  2012806
 gistMC.pressureScenario: well  120  of  277 :  2119704
 gistMC.pressureScenario: well  130  of  277 :  2116501
 gistMC.pressureScenario: well  140  of  277 :  2009788
 gistMC.pressureScenario: well  150  of  277 :  2121191
 gistMC.pressureScenario: well  160  of  277 :  2064

In [0]:
filteredDF,orderedWellList=gi.summarizePPResults(scenarioDF,currentWellsDF,threshold=dPCutoff,nOrder=nWells,verbose=2)

39  disaggregationPlotPP: wells have a  0.5  psi pressure contribution in one scenario
 disaggregationPlotPP:  39  sorted
 disaggregationPlotPP:  238  minimally-contributing wells sorted


In [0]:
disaggregationDF=gi.prepDisaggregationPlot(filteredDF,orderedWellList,jitter=0.1,verbose=0)

In [0]:
diffRange=(min(gist.diffPPVec),max(gist.diffPPVec))
rtDF,mergedWellsDF = gi.prepRTPlot(selectedWellsDF,ignoredWellsDF,minYear,diffRange,eq,clipYear=False,verbose=1)

 prepRTPlot: years before earthquake to plot: -44.56947296372348
 prepRTPlot: # of must include wells : 2
 prepRTPlot: # of wells to add to forecast : 129
 prepRTPlot: # of wells with 0 BBL disposal : 153


In [0]:
winWellsDF,winInjDF=gi.getWinWells(filteredDF,currentWellsDF,injDF)
display(winWellsDF)

index,ID,InjectionWellId,UniqueWellIdentifier,UICNumber,APINumber,LeaseName,Operator,OperatorType,OperatorPrincipalCompany,OperatorPrincipalCompanyType,WellName,WellNumber,State,Basin,County,District,SRAOrSIR,B3InjectionType,B3InjectionStatus,RegulatoryInjectionType,PermittedMaxLiquidBPD,PermittedMaxLiquidPSIG,PermittedMaxGasMCFPerDay,PermittedMaxGasPSIG,PermittedCommercialStatus,PermittedIntervalTopFt,PermittedIntervalBottomFt,InjectionClass,PermitStage,PermittedWellDepthClassification,DaysApplicationHasBeenInReview,DaysToPermitApproval,PermitIsAmendment,PendingApplicationIsAmendment,PendingApplicationRequestedCommercialStatus,PendingApplicationRequestedIntervalTopFt,PendingApplicationRequestedIntervalBottomFt,PendingApplicationRequestedMaxLiquidBPD,PendingApplicationRequestedMaxLiquidPSIG,PendingApplicationRequestedMaxGasMCFPerDay,PendingApplicationRequestedMaxGasPSIG,PendingApplicationRequestedOperator,PendingApplicationRequestedOperatorType,PendingApplicationRequestedOperatorPrincipalCompany,PendingApplicationRequestedOperatorPrincipalCompanyType,PendingApplicationRequestedWellDepthClassification,CompletedWellDepthClassification,CompletionAndDrillingPermitStatus,StartDate,TotalVerticalDepthFt,MeasuredDepthFt,WellboreOrientation,IsOpenHole,SurfaceHoleElevationFt,SurfaceHoleLatitude,SurfaceHoleLongitude,SurfaceHoleGeographySource,SurveyLinesDescription,B3RecordAddedUTCDateTime,B3RecordUpdatedUTCDateTime,B3RecordDeletedUTCDateTime,Distances,DXs,DYs,DDRatio,YearsInjecting,EncompassingDay,EncompassingDiffusivity,EventID,TotalBBL
10,2081279,81279,10142010000081279,84368,42-227-34375,GUITAR ESTATE -B-,AMERICO ENERGY RESOURCES,E&P,AMERICO ENERGY RESOURCES,E&P,GUITAR ESTATE -B- 2D,2D,TX,Permian - Midland,Howard,08,"Knott 15km SIR, Stanton 15km SIR",Saltwater Disposal,Active - Liquid,Injection Into Productive Zone,5000.0,2200.0,0.0,null,Non Commercial,8700.0,10800.0,Class 2 - Injection of fluids associated with oil and natural gas production,Not in Permitting Process,Permian Deep,null,null,true,null,null,null,null,null,null,null,null,null,null,null,null,null,Permian Deep,Completed,1996-06-24,11000.0,10700.0,Vertical,false,2516.0,32.274845,-101.62872,Lat/Long from Well Surface Location,660' FNL/1980' FWL,2024-02-16 20:41:47,2024-12-24 12:54:35,null,22.072159998864933,-16.610217462162282,-14.535514606443533,0.3029900828958923,28.09034907597536,-9319.0,0.04373394999077591,texnet2024oqfb,1.4922723999999993E7
37,2105036,105036,10142010000105036,108636,42-173-32542,MIDLAND NATIONAL BANK '39' SWD,ENVIRONMENTAL DISPOSAL SYSTEMS,E&P,DIAMONDBACK ENERGY,E&P,MIDLAND NATIONAL BANK '39' SWD 1,1,TX,Permian - Midland,Glasscock,08,null,Saltwater Disposal,Active - Liquid,Injection Into Non-Productive Zone,50000.0,5000.0,0.0,null,Commercial,11458.0,12496.0,Class 2 - Injection of fluids associated with oil and natural gas production,Not in Permitting Process,Permian Deep,null,55.0,true,null,null,null,null,null,null,null,null,null,null,null,null,null,Permian Deep,Completed,2015-02-01,13000.0,12496.0,Vertical,true,2609.0,31.958216,-101.75038,Lat/Long from Well Surface Location,1004' FSL & 484' FEL,2024-02-16 20:41:47,2024-12-24 12:54:35,null,21.31931936880207,-5.169505377376624,20.68307625921069,0.5036648608138964,9.483915126625599,-2586.0,0.12084958993836883,texnet2024oqfb,2.1987434000000004E7
41,2112104,112104,10142010000112104,115144,42-227-38978,LIMEQUEST 6 SWD,OXYROCK OPERATING,null,OCCIDENTAL PETROLEUM,E&P,LIMEQUEST 6 SWD 1D,1D,TX,Permian - Midland,Howard,08,"Knott 15km SIR, Stanton OLRP Outer - 9.08km",Saltwater Disposal,Active - Liquid,Injection Into Non-Productive Zone,30000.0,5000.0,0.0,null,Non Commercial,10150.0,13000.0,Class 2 - Injection of fluids associated with oil and natural gas production,Not in Permitting Process,Permian Deep,null,52.0,false,null,null,null,null,null,null,null,null,null,null,null,null,null,Permian Deep,Completed,2017-07-20,13500.0,11388.0,Vertical,true,2513.0,32.2809,-101.67925,Lat/Long from Well Surface Location,1980'

In [0]:

scenarioTSRDF,dPTimeSeriesR,wellIDsR,dayVecR = gist.runPressureScenariosTimeSeries(eq,winWellsDF,winInjDF,verbose=verb)

runPressureScenariosTimeSeries time axis information - nt: 1470 ; ot: 5450.0 ; dt: 10.0  earthquake index:  1448
runPressureScenariosTimeSeries r2 min/max:  6544256.262167588 2910336838.7003503
runPressureScenariosTimeSeries TSOver4TT min/max:  0.5247810359721807 46.659247071383
runPressureScenariosTimeSeries ppp min/max:  3434301.580927738 135794125617.86737
runPressureScenariosTimeSeries durations min/max:  864000.0 1270080000.0
runPressureScenariosTimeSeries epp min/max:  0.0 5.338508101352091
runPRessureScenariosTimeSeries time step  10  of  1470
runPRessureScenariosTimeSeries time step  20  of  1470
runPRessureScenariosTimeSeries time step  30  of  1470
runPRessureScenariosTimeSeries time step  40  of  1470
runPRessureScenariosTimeSeries time step  50  of  1470
runPRessureScenariosTimeSeries time step  60  of  1470
runPRessureScenariosTimeSeries time step  70  of  1470
runPRessureScenariosTimeSeries time step  80  of  1470
runPRessureScenariosTimeSeries time step  90  of  1470
run

In [0]:
totalPPQuantilesDF=gi.prepTotalPressureTimeSeriesPlot(dPTimeSeriesR,dayVecR,nQuantiles=11,epoch=pd.to_datetime('1970-01-01'),verbose=1)
totalPPSpaghettiDF=gi.prepTotalPressureTimeSeriesSpaghettiPlot(dPTimeSeriesR,dayVecR,gist.diffPPVec,epoch=pd.to_datetime('1970-01-01'),verbose=1)
print(totalPPQuantilesDF)

prepTotalPressureTimeSeriesPlot: deltaPP.shape= (39, 200, 1470)  dayVec.shape= (1470,)
prepTotalPressureTimeSeriesPlot: totalDeltaPP.shape= (200, 1470)
prepTotalPressureTimeSeriesPlot: quantiles: [0.0, 10.1, 20.1, 30.2, 40.2, 50.3, 59.8, 69.8, 79.9, 89.9, 100.0]
prepTotalPressureTimeSeriesSpaghettiPlot: deltaPP.shape= (39, 200, 1470)  dayVec.shape= (1470,)
prepTotalPressureTimeSeriesSpaghettiPlot: totalDeltaPP.shape= (200, 1470)
        DeltaPressure     Days  Realization  Percentile  Ordering       Date
0            0.000000   5450.0            0         0.0       0.0 1984-12-03
1            0.000000   5460.0            0         0.0       0.0 1984-12-13
2            0.000000   5470.0            0         0.0       0.0 1984-12-23
3            0.000000   5480.0            0         0.0       0.0 1985-01-02
4            0.000000   5490.0            0         0.0       0.0 1985-01-12
...               ...      ...          ...         ...       ...        ...
293836       4.746953  18510

In [0]:
allPPQuantilesDF,allPPSpaghettiDF=gi.getPerWellPressureTimeSeriesSpaghettiAndQuantiles(dPTimeSeriesR,dayVecR,gist.diffPPVec,wellIDsR,nQuantiles=11,epoch=pd.to_datetime('01-01-1970'))

In [0]:
wellPressureDict=gi.prepPressureAndDisposalTimeSeriesPlots(allPPQuantilesDF,allPPSpaghettiDF,winWellsDF,winInjDF,orderedWellList[:-1],verbose=0)

In [0]:
# Calculate sensitivities to different parameters
sensitivityDF,sensitivitySumDF = gist.getPressureSensitivity(winInjDF,winWellsDF,eq,verbose=2)
# I'm not getting negative pressures anymore
print(sensitivitySumDF)

getPressureSensitivity: ntaS min/max:  0.0009 0.0011
getPressureSensitivity: phiS min/max:  3.0 15.0
getPressureSensitivity: hMS min/max:  60.96 609.6
getPressureSensitivity: alphavS min/max:  1.07e-09 1.08e-09
getPressureSensitivity: betaS min/max:  3.59e-10 3.61e-10
getPressureSensitivity: kMDS min/max:  5.0 500.0
getPressureSensitivity: SS:  [[0.00371367]
 [0.00373197]
 [0.00375026]
 [0.00373197]
 [0.00373197]
 [0.00373197]
 [0.0036595 ]
 [0.00373197]
 [0.00380443]
 [0.00067854]
 [0.00373197]
 [0.00678539]
 [0.00369842]
 [0.00373197]
 [0.00373197]
 [0.00373167]
 [0.00373197]
 [0.00373227]
 [0.00373197]
 [0.00373197]
 [0.00373197]]
getPressureSensitivity: TS:  [[8.34524852e-04]
 [8.38635812e-04]
 [8.42746772e-04]
 [9.31817569e-04]
 [8.38635812e-04]
 [7.62396193e-04]
 [8.38635812e-04]
 [8.38635812e-04]
 [8.38635812e-04]
 [1.52479239e-04]
 [8.38635812e-04]
 [1.52479239e-03]
 [8.38635812e-04]
 [8.38635812e-04]
 [8.38635812e-04]
 [8.38635812e-04]
 [8.38635812e-04]
 [8.38635812e-04]
 [1.6

In [0]:
# This is the date to calcaulte THdPdT0
futureEQ=eq.copy()
futureEQ.loc['Origin Date']=pd.to_datetime('2026-9-1')
print(eq,futureEQ)
rateDF = gist.getTHdPdT0(winWellsDF,winInjDF,eq,futureEQ,2)

SeismicEventId                                      2481836
DataSource                        TexNet Earthquake Catalog
DataSourceUrl                                          None
EventID                                      texnet2024oqfb
EventTimeUtc                            2024-07-27 02:11:07
EventTimeInLocalTimeZone               2024-07-26T21:11:07Z
EventTimeZone                                           CST
EventType                                        Earthquake
DepthKm                                            8.837891
DepthErrorKm                                       0.636678
Magnitude                                           1.64016
MagnitudeError                                          NaN
MagnitudeType                                    ml(texnet)
Location                                      Western Texas
Status                                                final
Latitude                                          32.144165
LatitudeError                           

/Workspace/Users/bill.curry@exxonmobil.com/GIST/lib/gistMC.py:2201: RuntimeWarning: divide by zero encountered in divide
  Q_new=(dpdt * futureInjSec / OneOver4piKappaH) / expTerm


###Output

In [0]:
gist.writeRealizations(initialRunIntervalPath+'PorePressureRealizations.csv')

In [0]:
scenarioDF.to_csv(initialRunIntervalPath+'scenarios.csv')

In [0]:
filteredDF.to_csv(initialRunIntervalPath+'filteredScenarios.csv')
pd.Series(data=orderedWellList).to_csv(initialRunIntervalPath+'wellOrder.csv')

In [0]:
mergedWellsDF.to_csv(initialRunIntervalPath+'RTwells.csv')
rtDF.to_csv(initialRunIntervalPath+'RTDF.csv')

In [0]:
disaggregationDF.to_csv(initialRunIntervalPath+'disaggregation.csv')

In [0]:
totalPPQuantilesDF.to_csv(initialRunIntervalPath+'totalPPQuantiles.csv')
totalPPSpaghettiDF.to_csv(initialRunIntervalPath+'totalPPSpaghetti.csv')

In [0]:
i=0
for wellDictKey, wellDictValue in wellPressureDict.items():
  wellID=wellDictValue['WellInfo']['ID'].to_list()[0]
  wellFilePrefix='/perWell/well_'+str(i)+'_'
  wellDictValue['PPQuantiles'].to_csv(initialRunIntervalPath+wellFilePrefix+'PPQuantiles.csv')
  wellDictValue['Disposal'].to_csv(initialRunIntervalPath+wellFilePrefix+'Disposal.csv')
  wellDictValue['Spaghetti'].to_csv(initialRunIntervalPath+wellFilePrefix+'Spaghetti.csv')
  wellDictValue['WellInfo'].to_csv(initialRunIntervalPath+wellFilePrefix+'WellInfo.csv')
  print('well',i,', ID:',wellID,' completed')
  i=i+1

well 0 , ID: 2121094  completed
well 1 , ID: 2121271  completed
well 2 , ID: 2123938  completed
well 3 , ID: 2117885  completed
well 4 , ID: 2111946  completed
well 5 , ID: 2109094  completed
well 6 , ID: 2116501  completed
well 7 , ID: 2112288  completed
well 8 , ID: 2081404  completed
well 9 , ID: 2114817  completed
well 10 , ID: 2112104  completed
well 11 , ID: 2111358  completed
well 12 , ID: 2121191  completed
well 13 , ID: 2113898  completed
well 14 , ID: 2111690  completed
well 15 , ID: 2118993  completed
well 16 , ID: 2056720  completed
well 17 , ID: 2121195  completed
well 18 , ID: 2120454  completed
well 19 , ID: 2117633  completed
well 20 , ID: 2112172  completed
well 21 , ID: 2081279  completed
well 22 , ID: 2121203  completed
well 23 , ID: 2112058  completed
well 24 , ID: 2105036  completed
well 25 , ID: 2120859  completed
well 26 , ID: 2064982  completed
well 27 , ID: 2119049  completed
well 28 , ID: 2120980  completed
well 29 , ID: 2123953  completed
well 30 , ID: 212171

In [0]:
scenarioTSRDF.to_csv(initialRunIntervalPath+'materialScenariosR.csv')
np.savez_compressed(initialRunIntervalPath+'timeSeriesR.npz', deltaPP=dPTimeSeriesR,dayVec=dayVecR,wellIDs=wellIDsR)

In [0]:
sensitivityDF.to_csv(initialRunIntervalPath+'sensitivity.csv')
sensitivitySumDF.to_csv(initialRunIntervalPath+'sensitivitySum.csv')

In [0]:
rateDF.to_csv(initialRunIntervalPath+'rates.csv')

#3. Correct Data

##3.1 Export Disposal Data

In [0]:
selectedWellsDF.to_csv(disposalPath+'selectedWells.csv')
ignoredWellsDF.to_csv(disposalPath+'ignoredWells.csv')
allWellsDF=pd.concat([selectedWellsDF,ignoredWellsDF])
allWellsDF.to_csv(disposalPath+'allInZoneWells.csv')
injDF.to_csv(disposalPath+'inj.csv')

In [0]:
# End Year of time series forecast - I don't think that I need this now.
#EndYear=2030
# Rerun select wells with an updated earthquake time set to the last day of EndYear
#forecastEQ=eq.copy()
#forecastEQ['Origin Date']=pd.to_datetime('12-31-'+str(EndYear))
#selectedForecastWellsDF,ignoredForecastWellsDF,injDF=gist.findWells(forecastEQ,PE=False,verbose=verb)
##selectedWellsDF.to_csv(disposalPath+'forecastSelectedWells.csv')
#ignoredWellsDF.to_csv(disposalPath+'forecastIgnoredWells.csv')
#allWellsDF=pd.concat([selectedWellsDF,ignoredWellsDF])
#allWellsDF.to_csv(disposalPath+'forecastAllInZoneWells.csv')
#injDF.to_csv(disposalPath+'forecastPastInj.csv')

##3.2 Import Updated Disposal Data

In [0]:
# I need to merge the updated disposal information with the prior stuff
# Load updated injection file
# Well comparisons
# Get list of well IDs in the updated injection file
# Get list of well IDs in updated well file
# Check to see overlap in well IDs
#    updated vs. prior selected wells
#    updated vs. prior ignored wells
# FilteredIgnored =  prior ignored wells - (wells in updated file and prior ignored wells)
# Merge FilteredIgnored with with updated wells
# Load injection
#    Check for time sampling of injection - all wells must be the same
#    Get overall time vector - convert to days
#    Loop over wells in well file
#      Get all disposal data for well ID
#      Convert date values to days
#      
#mergedInjFile=